In [1020]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [1021]:
#Importing data 
scoredf = pd.read_csv('creditscore.csv')
scoredf.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [1022]:
#Removing unwanted characters
scoredf = scoredf.replace('_', '', regex=True)

In [1023]:
#Checking for duplicates
scoredf['ID'].duplicated().sum()

0

In [1024]:
scoredf['Customer_ID'].duplicated().sum()

87500

In [1025]:
#Converting datatypes
scoredf['Age'] = scoredf['Age'].astype(int)
scoredf['Annual_Income'] = scoredf['Annual_Income'].astype(float)
scoredf['Monthly_Balance'] = scoredf['Monthly_Balance'].astype(float)
scoredf['Num_of_Delayed_Payment'] = scoredf['Num_of_Delayed_Payment'].astype(float)


In [1026]:
#Removing outliers
scoredf['Age'] =  scoredf['Age'][(scoredf['Age'] >= 0) & (scoredf['Age'] <= 100)]

In [1027]:
scoredf['Age'].max()

100.0

In [1028]:
#Defining functions
def replace_with_mode(df, group_col, target_col , new_col_name):
    mode_df = df.groupby(group_col)[target_col].agg(lambda x: x.mode().iloc[0] if not x.mode().iloc[0] == '' else np.nan ).reset_index()
    df = df.merge(mode_df, on=group_col, suffixes=('', '_mode'))
    df.rename(columns={f'{target_col}_mode': new_col_name}, inplace=True)
    df.drop(columns=[target_col], inplace=True)
    return df

In [1029]:
#Filling missing values
user_mean_ages = scoredf.groupby('Customer_ID')['Age'].mean().round()
df_merged = scoredf.merge(user_mean_ages.rename('Mean_Age'), on='Customer_ID')
df_merged['Age'] = df_merged['Age'].fillna(df_merged['Mean_Age'])

scoredf = df_merged.drop(columns=['Mean_Age'])

In [1030]:
#Replacing with mode
num_credit = scoredf.groupby('Customer_ID')['Num_Credit_Card'].min().reset_index()
mode_credit = scoredf.merge(num_credit, on='Customer_ID').rename(columns={'Num_Credit_Card_y': 'num_credit_cards'})
mode_credit.drop(columns=['Num_Credit_Card_x'], inplace=True)

In [1031]:
#Replacing with mode
mode_credit = replace_with_mode(mode_credit, 'Customer_ID', 'Num_of_Loan' , 'num_of_loan')
mode_credit = replace_with_mode(mode_credit, 'Customer_ID', 'Num_Bank_Accounts' , 'num_bank_accounts')
mode_credit = replace_with_mode(mode_credit, 'Customer_ID', 'Interest_Rate' , 'intrest_rate')
mode_credit = replace_with_mode(mode_credit, 'Customer_ID', 'Occupation' , 'occupation')
mode_credit = replace_with_mode(mode_credit, 'Customer_ID', 'Monthly_Inhand_Salary' , 'monthly_inhand_salary')


In [1032]:
#Removing missing values
mode_credit.dropna(subset=['occupation'], inplace=True)

In [1033]:
#Filling missing values
name = mode_credit['Name'].groupby(mode_credit['Customer_ID']).transform('first')
mode_credit['Name'] = mode_credit['Name'].fillna(name)


In [1034]:
mode_credit['Monthly_Balance'] = mode_credit['Monthly_Balance'].dropna().apply(lambda x: x if x >= 0 else 0)
mode_credit = mode_credit[mode_credit['Delay_from_due_date'] >= 0]
mode_credit = mode_credit[mode_credit['Num_of_Delayed_Payment'] >= 0]


In [1035]:
mode_credit.dropna(subset=['Monthly_Balance'], inplace=True)
mode_credit['Monthly_Balance'].isna().sum()

0

In [1036]:
mode_credit['Monthly_Balance'].max()

1602.0405189622518

In [1037]:
#Final dataframe
scoredf = mode_credit

# scoredf.columns = [col.lower() for col in scoredf.columns]

scoredf

,ID,Customer_ID,Month,Name,Age,SSN,Annual_Income,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,num_credit_cards,num_of_loan,num_bank_accounts,intrest_rate,occupation,monthly_inhand_salary
0,0x1602,CUS0xd40,January,Aaron Maashoh,23.0,821-00-0265,19114.12,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7.0,...,80.41529543900253,HighspentSmallvaluepayments,312.494089,Good,4,4,3,3,Scientist,1824.843333
2,0x1604,CUS0xd40,March,Aaron Maashoh,23.0,821-00-0265,19114.12,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7.0,...,81.699521264648,LowspentMediumvaluepayments,331.209863,Good,4,4,3,3,Scientist,1824.843333
3,0x1605,CUS0xd40,April,Aaron Maashoh,23.0,821-00-0265,19114.12,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4.0,...,199.4580743910713,LowspentSmallvaluepayments,223.451310,Good,4,4,3,3,Scientist,1824.843333
5,0x1607,CUS0xd40,June,Aaron Maashoh,23.0,821-00-0265,19114.12,"Auto Loan, Credit-Builder Loan, Personal Loan,...",8,4.0,...,62.430172331195294,!@9#%8,340.479212,Good,4,4,3,3,Scientist,1824.843333
6,0x1608,CUS0xd40,July,Aaron Maashoh,23.0,821-00-0265,19114.12,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,8.0,...,178.3440674122349,LowspentSmallvaluepayments,244.565317,Good,4,4,3,3,Scientist,1824.843333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,0x25fe8,CUS0x942c,March,Nicks,25.0,078-73-5990,39628.99,"Auto Loan, and Student Loan",20,6.0,...,140.58140274528395,HighspentMediumvaluepayments,410.256158,Poor,6,2,4,7,Mechanic,3359.415833
99995,0x25fe9,CUS0x942c,April,Nicks,25.0,078-73-5990,39628.99,"Auto Loan, and Student Loan",23,7.0,...,60.97133255718485,HighspentLargevaluepayments,479.866228,Poor,6,2,4,7,Mechanic,3359.415833
99996,0x25fea,CUS0x942c,May,Nicks,25.0,078-73-5990,39628.99,"Auto Loan, and Student Loan",18,7.0,...,54.18595028760385,HighspentMediumvaluepayments,496.651610,Poor,6,2,4,7,Mechanic,3359.415833
99997,0x25feb,CUS0x942c,June,Nicks,25.0,078-73-5990,39628.99,"Auto Loan, and Student Loan",27,6.0,...,24.02847744864441,HighspentLargevaluepayments,516.809083,Poor,6,2,4,7,Mechanic,3359.415833


In [1038]:
scoredf.to_csv('credit.csv', index=False)